#Centralized training: define the upper bound

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0, '/content/drive/MyDrive/FL2022/Centralized')

In [ ]:
from options import args_parser
from utils import get_dataset, exp_details, plotFigure, get_n_params
from model_RN50 import ResNet50
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
sys.argv=['',
          '--lr=0.0001',
          '--epochs=60',
          '--norm=group_norm',
          '--local_bs=32',
          '--optimizer=adam',
          '--dataset=cifar',
          '--momentum=0.9', 
          '--loss=CrossEntropyLoss',
          '--gpu=/device:GPU:0']
args=args_parser()

In [ ]:
#DATASET
train_set, test_set, user_groups = get_dataset(args)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=args.local_bs, shuffle=True, num_workers=2) 
test_loader = torch.utils.data.DataLoader(test_set, batch_size=args.local_bs, shuffle=False, num_workers=2)

##Model

In [ ]:
resnet = ResNet50(args.norm)
resnet = resnet.to(device)
get_n_params(resnet)

##Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
if (args.optimizer == "adam"):
  optimizer = optim.Adam(resnet.parameters(), lr=args.lr) #ADAM optimizer
else: #SGD optimizer
  optimizer = optim.SGD(resnet.parameters(), lr=args.lr,
                        momentum=args.momentum, weight_decay=args.wd) 

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

##Train Function

In [ ]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    resnet.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (X_train, y_train) in enumerate(train_loader):
        X_train, y_train = X_train.to(device), y_train.to(device)
        optimizer.zero_grad()
        y_pred = resnet(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = y_pred.max(1)
        total += y_train.size(0)
        correct += predicted.eq(y_train).sum().item()
    
    train_loss_avg = train_loss/(batch_idx+1)
    train_acc_avg = correct/total
    print(f'TRAIN -> Epoch: {epoch:2} loss: {train_loss_avg:10.8f} accuracy: {100.*correct/total:7.3f}% ({correct}/{total})')

  
    return train_loss_avg, train_acc_avg

##Test Function

In [ ]:
def test(epoch):
    resnet.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, targets) in enumerate(test_loader):
            data, targets = data.to(device), targets.to(device)
            outputs = resnet(data)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    test_loss_avg = test_loss/(batch_idx+1)
    test_acc_avg = correct/total
    print(f'TEST -> Epoch: {epoch:2} loss: {test_loss/(batch_idx+1):10.8f} accuracy: {100.*correct/total:7.3f}% ({correct}/{total})')

    return test_loss_avg, test_acc_avg

In [ ]:
# [epoch, train_loss, test_loss, train_acc, test_acc]
res_list_M1 = [] 

##Main:

In [ ]:
for epoch in range(1,args.epochs+1):

    train_loss, train_acc = train(epoch)
    test_loss, test_acc = test(epoch)

    res_list_M1.append([epoch, train_loss, test_loss, train_acc, test_acc])

    scheduler.step()

##Save and Plots

In [ ]:
#Show and Save the Results 
fileName = f"{args.model}_{args.norm}_{args.optimizer}_lr_{args.lr}_bs{args.local_bs}_epochs_{args.epochs}.csv"
#creating the data frame
df = pd.DataFrame(res_list_M1, columns = ['epoch', 'train_loss', 'test_loss', 'train_acc', 'test_acc'])
df.to_csv('/content/drive/MyDrive/FL2022/Centralized/ResultCentralized/'+fileName, encoding='utf-8')
#plot
plotFigure(args, df)
